# Tensor Internals

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

Los _floating point numbers_ son la forma en la que las redes manejan la informacion.

La transformacion de un tipo de data a otro es normalmente aprendida por la red en etapas. 

Las representaciones son numeros que capturan la estructua de la data. 

Antes es importante entender como PyTorch maneja y guarda data - como input, representacion intermedia, y output.

### Tensores

La estructura de datos fundamental en PyTorch es el tensor. En este caso un tensor se refiere a una generalizacion de vectores y matrices a un numero arbitrario de dimensiones. Otro termino para el mismo son _arreglos multidimensionales_. La dimensionalidad de un tensor coincide con el numero de indices que se usan para referirse a un valor escalar dentro del tensor.

In [2]:
zero_d = torch.tensor(1)
one_d = torch.tensor([1,2,3,4,5])

two_d = torch.tensor([[1,2,3],
                     [4,5,6]])

three_d = torch.ones((28,28,3))

## Comparacion con NumPy

PyTorch provee buena interoperabilidad con NumPy. Esto provee integracion de primera clase con el resto del stack cientifico de Python: `SciPy`, `Scikit-learn`, `Pandas`, `Matplotlib`, etc.

Es como NumPy en esteroides. Tiene la habilidad de realizar operaciones sumamente rapidas sobre GPUs, distribuir operaciones a traves de muchos dispositivos o maquinas y mantiene un registro de las computaciones realizadas.

## Objetivos

1. Introducir tensores en PyTorch.
2. Manipular tensores en PyTorch.
3. Representacion de data en memoria.
4. Como se realizan operaciones en tensores de tamanios arbitrarios en tiempo constatne.
5. NumPy interoperability y aceleracion en GPUs.

# Fundamentos del Tensor

* Un arreglo
* Una estructua de datos que guarda una coleccion de numeros accesibles individualmente usando un indice.

In [3]:
a = [1.0, 2.0, 1.0]

In [4]:
a[0]

1.0

In [5]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

Usar listas para representar vectores de numeros como coordenadas de una linea es suboptimo:
* Los numeros en Python son objetos.
* Listas en Python estan hechas para colecciones sequenciales de objeos.
* El interpretador de Python es lento comparado a codigo compilado optimizado.

In [6]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [7]:
a[1]

tensor(1.)

In [8]:
float(a[1])

1.0

In [9]:
a[2] = 2.0
a

tensor([1., 1., 2.])

Las listas en Python son colecciones de objetos individualmente alocados en memoria.

Los tensores de PyTorch o los arrays de NumPy son views (normalmente) sobre bloques contiguos de memoria que contienen tipos numericos de C _unboxed_, no objetos de Python - 32-bit (4 bytes) `float` en este caso (**NOTA** Agregar figura). Esto significa que un tensor de 1D de 1,000,000 de floats requiere exactamente 4,000,000 bytes continuos para ser almacenado en memoria, mas un pequenio overhead para meta data (e.i. dimensiones, tipo numerico)

In [10]:
points = torch.FloatTensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [11]:
points[0, 1]

tensor(1.)

In [12]:
points[0]

tensor([4., 1.])

El ultimo output es otro tensor de 1D de tamanio 2 que contiene los valores de la primera fila del tensor `points`. 
1. Se aloco otro chunk de memoria?
2. Se copiaron los valores a el?
3. Retorno la nueva memoria envuelta en un nuevo objeto `tensor`?

## Tensores y Almacenamiento

La implementacion debajo del capo:

1. Los valores son alocados en bloques continuos de memoria, manejados por instancias de `torch.Storage`
2. Un `Storage` es un arreglo 1D de data numerica.
3. Un `Tensor` de PyTorch es una view sobre dicho `Storage` que es capaz de indexar dentro de ese storage usando un offset y pasos por dimension.
4. Varios tensores pueden indexar el mismo `Storage`.

**AGREGAR DIBUJO**

Podemos accesar el `storage` de un tensor usando la propiedad `.storage`

In [13]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [14]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

Aunque el tensor dice tener 3 filas y 2 columnas, en memoria es un arreglo continuo de tamanio 6. En este sentido el tensor, solo sabe como trasladar un par de indices a un espacio en memoria.

Podemos indexar la memoria de forma manual:

In [15]:
points_storage = points.storage()
points_storage[0]

4.0

No podemos indexar la memoria de un tensor 2D usando dos indices. El layout del almacenamiento siempre es 1D, a pesar de la dimensionalidad de cualquiera de los tensores que hagan referencia a el.

**IMPORTANTE** Cambiar el valor de un almacenamiento cambia el contenido de los tensores que le hacen referencia:

In [16]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

## Size, offset, stride

Para poder indexar a un `storage`, los tensores hacen uso de la siguiente informacion: size, offset y stride.

* size: una tupla indicando cuantos elementos a traves de cada dimension representa el tensor.
* offset: indice en el storage correspondiente al primer elemento en el tensor.
* stride: numero de elementos en el storage que se necesitan saltar para obtener el siguiente elemento a traves de cada dimension.

**AGREGAR DIBUJO**

In [17]:
points.shape

torch.Size([3, 2])

In [18]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()
second_point

tensor([5., 3.])

In [19]:
second_point.size()

torch.Size([2])

In [20]:
points.stride()

(2, 1)

In [21]:
points.stride()

(2, 1)

* El tensor resultante tiene un offset de 2 en el storage.
* size es una instancia de la clase `Size` que contiene un elemento, ya que el tensor es de 1D.
* stride es una tupla que indica el numero de elementos en el storage que tienen que ser saltados cuando el indice es incrementado por 1 en cada dimension.

Accesar el elemento $i, j$ en un tensor 2D resulta en accesar el elemento `storage_offset + stride[0] * i + stride[1] * j` en el storage.

La relacion entre `Tensor` y `Storage` hace que algunas operaciones, como transponer un tensor o extraer un sub-tensor, sean baratas ya que no requieren realocar memoria. En vez consisten en alocar una nueva instancia del objeto tensor con un diferente valor de size, storage, stride.

In [22]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.size()

torch.Size([2])

In [23]:
second_point.storage_offset()

2

In [24]:
second_point.stride()

(1,)

El sub-tensor tiene una dimension menos mientras sigue indexando el mismo storage que el tensor original. Esto tambien significa que modificar el sub-tensor va a tener un efecto en el original:

In [25]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point
second_point[0] = 10.0

In [26]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [27]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

Transpuesta: modificar el tensor que tiene puntos individuales en las filas y coordenadas $x,y$ en las columnas y voltearlo para que los puntos individuales esten a lo largo de las columnas

In [28]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [29]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [30]:
id(points.storage()) == id(points_t.storage())

True

In [31]:
points.stride()

(2, 1)

In [32]:
points_t.stride()

(1, 2)

In [33]:
x

NameError: name 'x' is not defined

In [ ]:
x = torch.ones((3, 4, 5))
x_t = x.transpose(0, 2)
x_t

In [ ]:
x_t.size()

In [34]:
x.stride()

NameError: name 'x' is not defined

In [35]:
x_t.stride()

NameError: name 'x_t' is not defined

Un tensor cuyos valores estan representados en el storage empezando desde la ultima dimension en adelante (e.i. moviendose a lo largo de las filas para un tensor 2D) se define como `continuo`. Los tensores `continuos` son convenientes porque podemos visitarlos en orden de forma eficiente sin saltar en el storage.

In [36]:
x.is_contiguous()

NameError: name 'x' is not defined

In [37]:
x_t.is_contiguous()

NameError: name 'x_t' is not defined

## Tipos Numericos

* Que tipos numericos podemos almacenar en un `Tensor`?
* El argumento `dtype` a los constructores del tensor (funciones como `tensor`, `zeros`, `ones`) especifica el tipo de data numerico que va a almacenar el tensor.
* El tipo de dato especifica los posibles valores del tensor (enteros vs. floating point numbers) y el numero de bytes por valor.

### Posibles tipos de datos

* `torch.float32` o `torch.float`: 32-bit floating point
* `torch.float64` o `torch.double`: 64-bit double precision floating point
* `torch.float16` o `torch.half`: 16-bit, half precision floating point
* `torch.int8`: signed 8-bit integers
* `torch.uint8`: unsigned 8-bit integers
* `torch.int16` o `torch.short`: signed 16-bit integers
* `torch.int32` o `torch.int`: signed 32-bit integers
* `torch.int64` o `torch.long`: signed 65-bit integers

**Task:**
* Cual es el rango de valores que cada uno de estos tipos puede representar?
* Cual es la diferencia entre un `int` y un `uint`?
* Cual es el tipo default que utiliza base python? NumPy? PyTorch?

In [38]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [39]:
short_points.dtype

torch.int16

In [40]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [41]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

Under the hood ambos metodos (`type` y `to`) realizan el la misma revision y conversion si es necesaria. Pero el metodo `to` toma argumentos adicionales.

Podemos hacer cast de un tensor de un tipo a un tensor de otro tipo utilizando el metodo `type`

In [42]:
# randn inicializa los elementos del tensor a numeros aleatorios entre 0 y 1
points = torch.randn(10, 2)
short_points = points.type(torch.short)

## Indexando Tensores

Vimos que `points[0]` retorna un tensor que contiene el punto 2D en la primera fila del tensor.

Que pasa si queremos obtener un tensor que contenga todos los puntos excepto el primero?

In [43]:
some_list = list(range(6))
some_list[:] # Todos los elementos de la lista
some_list[1:4] # Del elemento 1 inclusivo hasta el 4 exclusivo
some_list[1:] #  Del elemento 1 inclusivo hasta el final de la lista
some_list[:4] # Del primer elemento (indice 0) al 4 exclusivo
some_list[:-1] # del principio de la lista al ante penultimo
some_list[1:4:2] # Del elemento 1 inclusivo al elemento 4 exclusivo en pasos de 2

[1, 3]

Podemos hacer lo mismo en PyTorch con el beneficio adicional de poder utilizar range indexes para cada dimension del tensor.

In [44]:
points

tensor([[ 0.7712, -2.1432],
        [-0.4340,  3.4155],
        [ 0.8082,  0.3121],
        [-0.4220, -1.9289],
        [ 1.1187,  0.6022],
        [ 1.3363, -0.2809],
        [ 0.1922, -0.8501],
        [ 0.4619,  0.2344],
        [ 3.5001,  0.3173],
        [ 0.4216,  0.6355]])

In [45]:
points[1:]  # Todas las filas despues de la primera, implicitamente todas las columnas
points[1:, :] # Todas las filas despues de la primera, todas las columnas
points[1:, 0] # Todas las filas despues de la primera, la primera columna

tensor([-0.4340,  0.8082, -0.4220,  1.1187,  1.3363,  0.1922,  0.4619,  3.5001,
         0.4216])

In [46]:
points[1:, 0]

tensor([-0.4340,  0.8082, -0.4220,  1.1187,  1.3363,  0.1922,  0.4619,  3.5001,
         0.4216])

## Interop con NumPy

Los tensores de PyTorch pueden ser convertidos a arrays de NumPy y vice versa de forma eficiente. Dada la ubiquidad de NumPy en el ecosistema de data science de Python esto es algo importante. Esto nos permite aprovechar una enorme cantidad de funcionalidad que el ecosistema de Python a desarrollado alrededor del NumPy Array.

In [47]:
points = torch.ones(3, 4)
points_np = points.numpy()
type(points_np)

numpy.ndarray

In [48]:
points = torch.from_numpy(points_np)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

## Serializando tensores

Crear tensores de forma dinamica esta bien, pero si la data dentro de los tensores es valiosa, vamos a querer guardarla a un archivo y cargarla de nuevo en algun punto.

PyTorch utiliza `pickle` para serializar tensores

In [49]:
torch.save(points, "data/serialization-example/our_points.t")

FileNotFoundError: [Errno 2] No such file or directory: 'data/serialization-example/our_points.t'

In [50]:
with open("data/serialization-example/our_points.t", "wb") as f:
    torch.save(points, f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/serialization-example/our_points.t'

In [51]:
points = torch.load("data/serialization-example/our_points.t")

FileNotFoundError: [Errno 2] No such file or directory: 'data/serialization-example/our_points.t'

In [52]:
with open("data/serialization-example/our_points.t", "rb") as f:
    points = torch.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/serialization-example/our_points.t'

In [53]:
import h5py

f = h5py.File("data/serialization-example/our_points.hdf5", "w")
dset = f.create_dataset("coords", data=points.numpy())
f.close()

ModuleNotFoundError: No module named 'h5py'

* "coords" es una llave en el archivo HDF5. Podemos tener otras llevas e incluso llaves anidadas. 
* HDF5 nos permite indexar el dataset mientras esta en el disco y solo acceder los elementos que nos interesan.

Por ejemplo: Podemos cargar solo los ultimos dos puntos de nuestro dataset.

In [54]:
fs = h5py.File("data/serialization-example/our_points.hdf5", "r")
dset = fs["coords"]

last_points = dset[1:]
last_points

NameError: name 'h5py' is not defined

En este caso la data no fue cargada cuando el archivo se abrio o el dataset fue requerido. En vez, la data se mantuvo en disco hasta que solicitamos las ultimas dos filas. 

En ese punto, `h5py` acceso regreso un objeo similar a un array de NumPy que encapsula esa region del dataset y se comporta igual que un array de NumPy y tiene el mismo API. 

In [55]:
last_points = torch.from_numpy(last_points)
f.close()

NameError: name 'last_points' is not defined

In [56]:
last_points

NameError: name 'last_points' is not defined

## Moviendo tensores al GPU

Todos los tensores `Torch` pueden ser transferidos a un GPU para realizar computaciones en paralelo. Todas las operaciones que van a ser realizadas sobre el tensor seran llevadas a cabo usando rutinas especificas al GPU que vienen con PyTorch.

Adicional al `dtype`, un `Tensor` de PyTorch tambien tiene el argumento de `device`, que establece donde en la computadora se va a poner la data del tensor.

In [57]:
import torch

In [58]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]).to(device)

In [59]:
points = 2 * points
points_gpu = 2 * points.to(device)

El tensor `points_gpu` no regrese al CPU una vez los resultados han sido calculados. Lo que paso arriba es lo siguiente:

1. El tensor `points` fue copiado al GPU
2. Un nuevo tensor fue alocado en el GPU y usado para guardar los resultados de la multiplicacion
3. Un handle al tensor GPU fue retornado

In [60]:
points_gpu = points_gpu + 4

Si agregamos un constante al resultado, la suma fue realizada en el GPU, la informacion no fluye al CPU (excepto si intentamos acceder el tensor ei. `print()`). Para mover el tensor de vuelta al CPU necesitamos proveer el argumento `cpu` al metodo `to`.

In [61]:
points_cpu = points_gpu.to(device='cpu')

## El Tensor API

[PyTorch Docs](http://pytorch.org/docs)

La mayoria de operaciones sobre y entre tensores estan disponibles bajo el modulo `torch` y pueden ser llamadas como metodos del objeto `tensor`.

La operaciones sobre tensores estan divididas en grupos:

* Creation ops: funciones para construir un tensor, como `ones` y `from_numpy`
* Indexing, slicing, joining, mutating ops: funciones para cambiar el `shape`, `strid` o los contenidos de un tensor, como `transpose`
* Math ops: funciones para manipular el contenido de un tensor a traves de computaciones:
    * Pointwise ops: funciones para obtener un nuevo tensor al aplicar una funcion a cada elemento de forma independiente. `abs` y `cos`
    * Reduction ops: funciones para calcular valores agregados al iterar sobre tensores. `mean`, `std` y `norm`
    * Comparison ops: funciones para evaluar predicados numericos sobre tensores. `equal` y `max`
    * Spectral ops: funciones para transformar y operar en frequencias. `stft` y `hamming_window`
    * Other ops: funciones especiales para operar sobre vectores, como `cross` o matrices como `trace`
    * BLAS y LAPACK ops: funciones que siguen las especificaciones BLAS (Basic Linear Algebra Subprograms) para operaciones escalares, vector-vector, matrix-vector, matrix-matrix
    * Random sampling: funciones para generar valores muestreando aleatoriamente de distribuciones de probabilidades. `randn` y `normal`
    * Serialization: funciones para guardar y cargar tensores como `load` y `save`
    * Parallelism: funciones para controlar el numero de threads para ejecucion paralela en el CPU, como `set_num_threads`

## Ejercicios

1. Crear un tensor de `list(range(9))` e indicar cual es el `size`, `offset`, y `strides`
    * Crear un tensor `b = a.view(3, 3)`. Cual es el valor de `b[1, 1]`
    * crear un tensor `c = b[1:, 1:]`. Cual es el `size`, `offset`, `strides`?
2. Escogan una operacion matematica como cosine o sqrt. Hay una funcion correspondiente en PyTorch?
    * Existe una version de esa operacion que opera `in-place`?

1. Crear un tensor 2D y luego agregar una dimension de tamanio 1 insertada en la dimension 0.
2. Eliminar la dimension extra que agrego en el tensor previo.
3. Crear un tensor aleatorio de forma $5x3$ en el intervalo $[3,7)$
4. Crear un tensor con valores de una distribucion normal ($\mu=0, \sigma=1$)
5. Recuperar los indices de todos los elementos no cero en el tensor `torch.Tensor([1,1,1,0,1])`.
6. Crear un tensor aleatorio de forma `(3,1)` y luego apilar cuatro copias horizontalmente.
7. Retornar el producto batch matrix-matrix de dos matrices 3D: (`a=torch.randn(3,4,5)`, `b=torch.rand(3,5,4)`)
8. Retornar el producto batch matrix-matrix de una matriz 3D y una matriz 2D: (`a=torch.rand(3,4,5)`, `b=torch.rand(5,4)`).

###  Ejercicio 1

In [62]:
list1 = list(range(9))

In [63]:
list_tensor = torch.FloatTensor(list1)
b = list_tensor.view(3,3)
b[1,1]

tensor(4.)

In [64]:
c = b[1:, 1:]
print(c.size())
print(c.stride())
print(c.storage_offset())

torch.Size([2, 2])
(3, 1)
4


### Ejercicio 2

In [65]:
d = torch.sin(c)

### Ejercicio 1


In [76]:
dim2 = torch.tensor([ [1, 2], 
                     [3, 4] ])
dim2.size()

dim2.unsqueeze_(0)
dim2.size()

torch.Size([1, 2, 2])

### Ejercicio 2

In [77]:
dim2.squeeze_(0)
dim2.size()

torch.Size([2, 2])

### Ejercicio 3

In [69]:
rand_tensor = (7-3)*torch.rand((5, 3)) + 3
rand_tensor

tensor([[4.4801, 5.8452, 4.2638],
        [5.3488, 3.2026, 6.5445],
        [6.6437, 6.9859, 5.0077],
        [5.6525, 3.8113, 5.6139],
        [5.3591, 5.6730, 6.7145]])

### Ejercicio 4

In [70]:
## la funcion randn devuelve un tensor con distribucion normal, media = 0 y stdev = 1

normal_tensor = torch.randn(5, 3, dtype=torch.double)
normal_tensor

tensor([[-3.7988e-01, -2.6896e-04,  8.8391e-01],
        [-4.3454e-01, -8.6644e-01, -1.6903e+00],
        [ 2.5089e+00,  7.8132e-01, -4.1374e-01],
        [ 1.0722e-01,  7.8682e-01,  1.3933e+00],
        [-8.6952e-01,  1.0391e+00, -7.9935e-01]], dtype=torch.float64)

### Ejercicio 6

In [71]:
tensor = torch.rand(3, 1)
torch.flatten(tensor)

tensor([0.5713, 0.0033, 0.7714])

### Ejercicio 7

In [72]:
a=torch.randn(3,4,5)
b=torch.rand(3,5,4)
multi = torch.matmul(a,b)
multi

tensor([[[ 2.3128,  1.5864,  0.6850,  1.9467],
         [ 2.0284,  2.2344,  1.7210,  0.4476],
         [ 0.2817, -0.8547, -1.2627, -0.4490],
         [-1.8043, -3.1152, -3.1229, -3.2066]],

        [[-0.1690, -0.0966,  0.5237,  0.2671],
         [ 0.9291, -0.3143,  0.4690,  0.3614],
         [ 1.4414,  0.9719, -0.0781,  1.2924],
         [ 1.3960,  0.8095,  1.6529,  1.6557]],

        [[ 0.2646,  0.5768, -0.0379,  0.6541],
         [ 0.2770,  0.2532,  0.8022, -0.0393],
         [ 0.3274,  1.3279,  1.9297,  0.6848],
         [-0.1136, -1.6971, -2.4872, -0.6490]]])

### Ejercicio 8

In [73]:
a=torch.rand(3,4,5)
b=torch.rand(5,4)
multi = torch.matmul(a,b)
multi

tensor([[[1.9396, 1.5361, 1.9218, 2.4033],
         [0.5749, 0.6239, 1.0310, 1.0033],
         [1.7999, 1.5376, 1.4630, 1.8588],
         [2.0157, 1.8235, 1.6263, 1.6339]],

        [[1.7583, 1.1762, 1.4926, 1.9861],
         [0.6595, 0.7603, 0.7893, 0.8147],
         [1.9499, 1.6419, 1.7749, 2.2120],
         [0.5485, 0.6910, 0.8036, 0.7875]],

        [[2.1878, 1.8484, 1.7818, 2.1072],
         [0.8368, 0.7382, 1.0302, 1.0082],
         [1.3083, 1.0079, 1.1657, 1.1899],
         [1.4290, 0.9161, 1.2912, 1.6744]]])